In [1]:
from langchain_community.llms import Ollama
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_community.embeddings import OllamaEmbeddings
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.document_loaders import PyPDFLoader

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
llm = Ollama(model='llama3.2')
embeddings = OllamaEmbeddings(model='nomic-embed-text')

In [4]:
loader = PyPDFLoader('t.pdf')
pages = loader.load_and_split()
store = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)
retriever = store.as_retriever()

In [5]:
template = """
Answer the question based only on the context provided.

Context: {context}

Question: {question}

** Do not provide any additional details.
"""

prompt = PromptTemplate.from_template(template)

def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

In [6]:
chain = (
  {
    'context': retriever | format_docs,
    'question': RunnablePassthrough(),
  }
  | prompt
  | llm
  | StrOutputParser()
)

In [8]:
question = 'What is cricket?'
print()
print(chain.invoke({'question': question}))
print()


A bat-and-ball sport played between two teams of eleven players each.

